In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import csv

In [ ]:
# load DSB coordinates
output_folder = 'output_0427_tiling600_2populationNonBypassingLEFs_GapMethod_TargetedLoading_run1'
with open(output_folder+'/DSB_boundary_coordinates.npy',"rb") as f:
    DSB_coordinates_np = np.load(f)
    boundary_coordinates_np = np.load(f)

In [ ]:
LEF_file = [filename for filename in os.listdir(output_folder) if filename.startswith("LEFcoordinates")]
time_list = [0, 4, 10, 20, 60, 90, 120] #in minutes
binsize = 5
for file in LEF_file:
    suffix = file.split('LEFcoordinates')[1]
    with open(output_folder+'/'+file,"rb") as f:
        LEF_coordiates = np.load(f)
        
    for time in time_list:
        LEFBedFileName = "LEFcount"+ suffix +'_' +str(time)+"min.bed"
        file_path = output_folder + '/'+ LEFBedFileName
        o = open(file_path, "w")
        o.close()
    start = 0
    count = 0 
    while start+binsize<= boundary_coordinates_np[-1]:
        for i in range(len(time_list)):
            left_extruders = LEF_coordiates[i*2,:]
            right_extruders = LEF_coordiates[i*2+1,:]
            test_1 = np.where((left_extruders-start)*(left_extruders-start-binsize)<=0,0.5,0) 
            test_2 = np.where((right_extruders-start)*(right_extruders-start-binsize)<=0,0.5,0) 
            score = np.sum(test_1+test_2) 
            line = "chr1\t "+str(start)+"\t "+str((start+binsize))+"\t region"+str(count)+"\t "+ str(score)+"\n"
            LEFBedFileName = "LEFcount"+ suffix +'_' +str(time_list[i])+"min.bed"
            file_path = output_folder + '/'+ LEFBedFileName
            with open(file_path, "a") as f:
                f.write(line)

        start += binsize
        count += 1